# Exercise : 서비스 이탈예측 데이터(Classification)

- 출처 : https://www.datamanim.com/dataset/03_dataq/typetwo.html#id3

- 데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
- 데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형
- X_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv
- y_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv
- X_test : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv
- y_test(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv

## 0. 데이터 로드 & 확인 (EDA)

In [30]:
import pandas as pd

X_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
X_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv")

In [2]:
X_train.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15799217,Zetticci,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,15748986,Bischof,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,15722004,Hsiung,543,France,Female,31,4,138317.94,1,0,0,61843.73
3,15780966,Pritchard,709,France,Female,32,2,0.00,2,0,0,109681.29
4,15636731,Ts'ai,714,Germany,Female,36,1,101609.01,2,1,1,447.73


In [3]:
y_train.head()

,CustomerId,Exited
0,15799217,0
1,15748986,0
2,15722004,0
3,15780966,0
4,15636731,0


## 1. 데이터 전처리

### 1-1. 불필요 칼럼 제거 및 처리
- 학습에 필요하지 않은 칼럼 제거

In [31]:
# 1-1. 불필요 칼럼 제거 및 처리
# 학습에 필요하지 않은 칼럼 제거

X_train = X_train.drop(["CustomerId", "Surname"], axis=1)
print(X_train.head())

X_test = X_test.drop(["CustomerId", "Surname"], axis=1)
print(X_test.head())

   CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          791   Germany  Female   35       7   52436.20              1   
1          705   Germany    Male   42       8  166685.92              2   
2          543    France  Female   31       4  138317.94              1   
3          709    France  Female   32       2       0.00              2   
4          714   Germany  Female   36       1  101609.01              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               0        161051.75  
1          1               1         55313.51  
2          0               0         61843.73  
3          0               0        109681.29  
4          1               1           447.73  
   CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          802    France  Female   60       3   92887.06              1   
1          602    France  Female   56       3  115895.22              3   
2          801    France  Female   3

- target 변수 따로 지정

In [32]:
target = y_train["Exited"]
print(target.head())
print(target.value_counts())

0    0
1    0
2    0
3    0
4    0
Name: Exited, dtype: int64
0    5175
1    1324
Name: Exited, dtype: int64


In [33]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      6499 non-null   int64  
 1   Geography        6499 non-null   object 
 2   Gender           6499 non-null   object 
 3   Age              6499 non-null   int64  
 4   Tenure           6499 non-null   int64  
 5   Balance          6499 non-null   float64
 6   NumOfProducts    6499 non-null   int64  
 7   HasCrCard        6499 non-null   int64  
 8   IsActiveMember   6499 non-null   int64  
 9   EstimatedSalary  6499 non-null   float64
dtypes: float64(2), int64(6), object(2)
memory usage: 507.9+ KB


In [34]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3501 entries, 0 to 3500
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      3501 non-null   int64  
 1   Geography        3501 non-null   object 
 2   Gender           3501 non-null   object 
 3   Age              3501 non-null   int64  
 4   Tenure           3501 non-null   int64  
 5   Balance          3501 non-null   float64
 6   NumOfProducts    3501 non-null   int64  
 7   HasCrCard        3501 non-null   int64  
 8   IsActiveMember   3501 non-null   int64  
 9   EstimatedSalary  3501 non-null   float64
dtypes: float64(2), int64(6), object(2)
memory usage: 273.6+ KB


### 1-2. 결측치 확인 및 처리 -> 결측치 없음

In [42]:
print(X_train.isnull().sum())
print(X_test.isnull().sum())

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64


### 1-3. 스케일링 및 인코딩

In [36]:
print("스케일링 전 \n", X_train.head()) # 스케일링 전 확인

스케일링 전 
    CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          791   Germany  Female   35       7   52436.20              1   
1          705   Germany    Male   42       8  166685.92              2   
2          543    France  Female   31       4  138317.94              1   
3          709    France  Female   32       2       0.00              2   
4          714   Germany  Female   36       1  101609.01              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               0        161051.75  
1          1               1         55313.51  
2          0               0         61843.73  
3          0               0        109681.29  
4          1               1           447.73  


In [39]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 수치형 칼럼 MinMaxScaler
n_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

scaler = MinMaxScaler()

X_train[n_cols] = scaler.fit_transform(X_train[n_cols])
X_test[n_cols] = scaler.transform(X_test[n_cols])

print("스케일링 후 \n ", X_train.head()) # 스케일링 후 확인


스케일링 후 
     CreditScore Geography  Gender       Age  Tenure   Balance  NumOfProducts  \
0        0.882   Germany  Female  0.229730     0.7  0.219962       0.000000   
1        0.710   Germany    Male  0.324324     0.8  0.699222       0.333333   
2        0.386    France  Female  0.175676     0.4  0.580223       0.000000   
3        0.718    France  Female  0.189189     0.2  0.000000       0.333333   
4        0.728   Germany  Female  0.243243     0.1  0.426235       0.333333   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               0         0.805365  
1          1               1         0.276566  
2          0               0         0.309224  
3          0               0         0.548461  
4          1               1         0.002181  


In [41]:
# 범주형 칼럼 LabelEncoder

c_cols = ['Geography', 'Gender']
for col in c_cols :
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

print(X_train.head())

   CreditScore  Geography  Gender       Age  Tenure   Balance  NumOfProducts  \
0        0.882          1       1  0.229730     0.7  0.219962       0.000000   
1        0.710          1       2  0.324324     0.8  0.699222       0.333333   
2        0.386          0       1  0.175676     0.4  0.580223       0.000000   
3        0.718          0       1  0.189189     0.2  0.000000       0.333333   
4        0.728          1       1  0.243243     0.1  0.426235       0.333333   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               0         0.805365  
1          1               1         0.276566  
2          0               0         0.309224  
3          0               0         0.548461  
4          1               1         0.002181  


## 2. 검증 데이터셋 분류

In [44]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, target, test_size = 0.2, random_state= 1)

print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(5199, 10) (1300, 10) (5199,) (1300,)


## 3. 모델 & 평가지표 비교

In [45]:
from sklearn.metrics import accuracy_score, roc_auc_score # 평가지표 라이브러리 호출

In [ ]:
# KNN



In [ ]:
# SVC 

In [52]:
# RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state = 1)
model_rf.fit(X_tr, y_tr)
pred_rf = model_rf.predict(X_val) # 등급을 예측


acc_rf = accuracy_score(y_val, pred_rf)
roc_rf = roc_auc_score(y_val, pred_rf)

print("accuracy score :", acc_rf) # model 기본값 -> 0.8561 정도
print("roc auc score :", roc_rf) # model 기본값 -> 0.701

accuracy score : 0.8561538461538462
roc auc score : 0.7019141372709873


In [55]:
pip install xgboost

     ---------------------------------------- 89.1/89.1 MB 7.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
# XGBClassifier

from xgboost import XGBClassifier
import xgboost as xgb

model_xg = xgb.XGBClassifier(random_state=1)
model_xg.fit(X_tr, y_tr)
pred_xg = model_xg.predict(X_val)

acc_xg = accuracy_score(y_val, pred_xg)
roc_xg = roc_auc_score(y_val, pred_xg)

print("accuracy score :", acc_xg) # model 기본값 -> 0.8569
print("roc auc score :", roc_xg) # model 기본값 -> 0.7234 

# 기본값 -> rf 보다 조금 더 높은 성능


accuracy score : 0.8569230769230769
roc auc score : 0.7234527390392853


In [ ]:
# from sklearn.model_selection import GridSearchCV

# def make_models(x_train, y_train):
#     param_grid = [{'C' : [2, 2.2, 2.4, 2.6, 2.8]}]
#     # param_grid = [{'penalty' : ['l1', 'l2'], 'C' : [2, 2.2, 2.4, 2.6, 2.8]}]
#     gs = GridSearchCV(estimator = LogisticRegression(), param_grid = param_grid,
#                     scoring = 'roc_auc', cv = 10)
#     result = gs.fit(x_train, y_train)
#     print(f'최적 점수: {gs.best_score_}')
#     print(f'최적 파라미터: {gs.best_params_}')

## 4. 최종 모델 선택